In [39]:
import pandas as pd
from metar import Metar
import numpy as np
from IOfuncs import *
import datetime as dt
import warnings
from json import JSONDecodeError
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFdr,SelectFpr,f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings('ignore')

In [40]:
def make_ml_data_row(taf_time, station, lat, lon, metar_path, glamp_path, hrrr_path, delay_hours = 2):
    if isinstance(metar_path, str):
        metar_path = read_metar(metar_path)
    metarDF = pd.DataFrame()
    glampDF = pd.DataFrame()
    hrrrDF = pd.DataFrame()    
    
    for time in range(-6, -delay_hours, 1):
        metar_at_time = get_metar_at_time(taf_time + dt.timedelta(hours = time), metar_path).T
        metarDF[f'metar {time}'] = metar_at_time
    
    work_time = dt.timedelta(hours=-delay_hours)
    glamp_data = get_glamp_at_time(taf_time + work_time, glamp_path, station, download=True)
    hrrr_data = get_hrrr_at_time(taf_time + work_time, hrrr_path, lat, lon, download=True)
    glamp_synoptic_offset = (taf_time.hour - delay_hours) % 6 - 1
    for time in range(-delay_hours, 7, 1):
        glampDF[f'glamp {time}'] = glamp_data.iloc[time + delay_hours + glamp_synoptic_offset]
        hrrrDF[f'hrrr {time}'] = hrrr_data.iloc[time + delay_hours]    
        
    
    df = pd.concat([metarDF, glampDF, hrrrDF])
    df.drop(['ftime', 'ftime_utc', 'model', 'runtime', 'runtime_utc', 'station', 'metar', 'peak_wind_time', 'valid', 'Unnamed: 0'], inplace=True)

    v = df.unstack().to_frame().sort_index(level=1).T
    v.columns = v.columns.map('_'.join)

    final = v.dropna(axis = 1)
    
    return final

In [41]:
taf_time = dt.datetime(year = 2021, month = 8, day = 21, hour = 18, minute = 0)

In [42]:
def make_ml_training_data_row(taf_time, station, lat, lon, metar_path, glamp_path, hrrr_path, asos5_path, delay_hours = 2, tplus_hours = 6):
    if isinstance(metar_path, str):
        metar_path = full_metar_list = read_metar(metar_path)
    if isinstance(asos5_path, str):
        asos5_path = read_metar(asos5_path)
    
    df = make_ml_data_row(taf_time, station, lat, lon, metar_path, glamp_path, hrrr_path, delay_hours = delay_hours)
    
    for i in range(tplus_hours):
        df[f'flight category {i}'] = get_conditions_from_asos(taf_time + dt.timedelta(hours = i), metar_path)
        time_series = pd.date_range(taf_time + dt.timedelta(hours = i), taf_time + dt.timedelta(hours = i+1), freq = '5T')
        verification_series = np.asarray(asos5_path.truncate(before = taf_time + dt.timedelta(hours = i), 
                                                             after = taf_time + dt.timedelta(hours = i+1))['conditions'])
        df[f'verification list {i}'] = [None]
        df[f'verification list {i}'][0] = verification_series
        

    return df

In [43]:
metar = read_metar('Data/BOS.csv')
asos5 = read_metar('Data/BOS_5min.csv')

In [44]:
%%time
make_ml_training_data_row(taf_time, 'kbos', 42.3656, -71.0096, metar, 'Data/GLAMP data/', 'Data/hrrr/', asos5)

CPU times: user 94.3 ms, sys: 0 ns, total: 94.3 ms
Wall time: 134 ms


,hrrr -1_DPT_1000mb,hrrr -2_DPT_1000mb,hrrr 0_DPT_1000mb,hrrr 1_DPT_1000mb,hrrr 2_DPT_1000mb,hrrr 3_DPT_1000mb,hrrr 4_DPT_1000mb,hrrr 5_DPT_1000mb,hrrr 6_DPT_1000mb,hrrr -1_DPT_2m_above_ground,...,flight category 1,verification list 1,flight category 2,verification list 2,flight category 3,verification list 3,flight category 4,verification list 4,flight category 5,verification list 5
0,292.8,293.5,292.2,292.0,291.8,294.0,293.5,292.2,293.2,293.2,...,3,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]",3,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]",3,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]",3,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]",3,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"


In [45]:
def make_ml_training_data_set(start_time, end_time, station, lat, lon, metar_path, glamp_path, hrrr_path, asos5_path, delay_hours = 2, frequency = '5H'):
    training_df = pd.DataFrame()
    time_series = pd.date_range(start_time, end_time, freq = frequency)
    if isinstance(metar_path, str):
        metar_path = read_metar(metar_path)
    if isinstance(asos5_path, str):
        asos5_path = read_metar(asos5_path)
    for time in tqdm(time_series):
        try:
            training_row = make_ml_training_data_row(time, station, lat, lon, metar_path, glamp_path, hrrr_path, asos5_path, delay_hours = delay_hours)
            training_df = pd.concat([training_df, training_row])
        except (FileNotFoundError, JSONDecodeError):
            continue

    training_df = training_df.fillna(-99999)
    return training_df

In [46]:
def prob_of_detection(predicted_results, actual_results, flight_cat):
    result_locations = np.where(actual_results==flight_cat)[0]
    num_predict = np.sum(predicted_results[result_locations]==flight_cat)
    return num_predict / len(result_locations)

def false_alarm_rate(predicted_results, actual_results, flight_cat):
    num_predict = np.sum(predicted_results==flight_cat)
    predict_locations = np.where(predicted_results==flight_cat)[0]
    predict_subset = predicted_results[predict_locations]
    actual_subset = actual_results.iloc[predict_locations]
    
    false_alarm_count = np.sum(predict_subset!=actual_subset)
    
    return false_alarm_count/num_predict

def critical_success_index(predicted_results, actual_results, flight_cat):
    num_predict = np.sum(predicted_results==flight_cat)
    predict_locations = np.where(predicted_results==flight_cat)[0]
    non_predict_locations = np.where(predicted_results!=flight_cat)[0]
    predict_subset = predicted_results[predict_locations]
    actual_subset = actual_results.iloc[predict_locations]
    actual_subset_compliment = actual_results.iloc[non_predict_locations]
    
    hits = np.sum(predict_subset==actual_subset)
    false_alarm_count = np.sum(predict_subset!=actual_subset)
    misses = np.sum(actual_subset_compliment==flight_cat)
    
    return hits / (hits + false_alarm_count + misses)

In [47]:
def prob_of_detection_nfv(predicted_results, actual_results):
    result_locations = np.where(actual_results!=3)[0]
    num_predict = np.sum(predicted_results[result_locations]!=3)
    return num_predict / len(result_locations)

def false_alarm_rate_nfv(predicted_results, actual_results):
    num_predict = np.sum(predicted_results!=3)
    predict_locations = np.where(predicted_results!=3)[0]
    predict_subset = predicted_results[predict_locations]
    actual_subset = actual_results.iloc[predict_locations]
    
    false_alarm_count = np.sum(predict_subset!=actual_subset)
    
    return false_alarm_count/num_predict

def critical_success_index_nfv(predicted_results, actual_results):
    num_predict = np.sum(predicted_results!=3)
    predict_locations = np.where(predicted_results!=3)[0]
    non_predict_locations = np.where(predicted_results==3)[0]
    predict_subset = predicted_results[predict_locations]
    actual_subset = actual_results.iloc[predict_locations]
    actual_subset_compliment = actual_results.iloc[non_predict_locations]
    
    hits = np.sum(predict_subset==actual_subset)
    false_alarm_count = np.sum(predict_subset!=actual_subset)
    misses = np.sum(actual_subset_compliment!=3)
    
    return hits / (hits + false_alarm_count + misses)

In [48]:
def data_split(data):
    y_keys = np.asarray([key for key in data if 'flight category' in key])
    val_keys = np.asarray([key for key in data if 'verification list' in key])
    X = data.drop(np.concatenate([y_keys, val_keys]), axis=1)
    y_list = data[y_keys]
    val_list = data[val_keys]
    
    return X, y_list, val_list


In [49]:
start_date = dt.datetime(year = 2020, month = 1, day = 1, hour = 0, minute = 0)
end_date = dt.datetime.now()
data = make_ml_training_data_set(start_date, end_date, 'kbos', 42.3656, -71.0096, 'Data/BOS.csv', 'Data/GLAMP data/', 
                                 'Data/hrrr/', 'Data/BOS_5min.csv', frequency = 'H')

100%|██████████████████████████████████████████████████████████████████████████| 24105/24105 [16:10:36<00:00,  2.42s/it]


In [ ]:
start_date_winter = dt.datetime(year = 2020, month = 12, day = 1, hour = 0, minute = 0)
end_date_winter = dt.datetime(year = 2021, month = 2, day = 28, hour = 23, minute = 0)
winter_data = make_ml_training_data_set(start_date_winter, end_date_winter, 'kbos', 42.3656, -71.0096, 'Data/BOS.csv',
                                        'Data/GLAMP data/', 'Data/hrrr/', 'Data/BOS_5min.csv', frequency = 'H')

In [ ]:
start_date_spring = dt.datetime(year = 2021, month = 3, day = 1, hour = 0, minute = 0)
end_date_spring = dt.datetime(year = 2021, month = 5, day = 31, hour = 23, minute = 0)
spring_data = make_ml_training_data_set(start_date_spring, end_date_spring, 'kbos', 42.3656, -71.0096, 'Data/BOS.csv', 
                                        'Data/GLAMP data/', 'Data/hrrr/', 'Data/BOS_5min.csv', frequency = 'H')

In [ ]:
def test_accuracy_metrics(classifier_rf, training_data, flight_cat):
    result_df = pd.DataFrame()
    (X, y_list, val_list) = data_split(training_data)
    for y in y_list:
        X_train, X_test, y_train, y_test = train_test_split(X, y_list[y], train_size=0.7, random_state=42)
        classifier_rf.fit(X_train, y_train)
        prob = classifier_rf.predict(X_test)
        result_df[y] = (prob_of_detection(prob, y_test, flight_cat), 
                        false_alarm_rate(prob, y_test, flight_cat), 
                        critical_success_index(prob, y_test, flight_cat))
        result_df.rename({0: 'POD', 1: 'FAR', 2: 'CSI'}, inplace=True)
    return result_df

In [ ]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs = -1)
classifier_nn = MLPClassifier(random_state=42)
#params determined via hyperparam tuning
n_estimators = [int(x) for x in np.linspace(200, 2000, 10)]
max_depth = [x for x in range(10, 120, 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
tuned_rf = RandomizedSearchCV(estimator = classifier_rf, param_distributions = random_grid, random_state = 42, n_jobs = -1)

In [ ]:
def test_accuracy_metrics_nfv(classifier_rf, training_data):
    result_df = pd.DataFrame()
    (X, y_list) = data_split(training_data)
    for y in y_list:
        X_train, X_test, y_train, y_test = train_test_split(X, y_list[y], train_size=0.7, random_state=42)
        classifier_rf.fit(X_train, y_train)
        prob = classifier_rf.predict(X_test)
        #result_df[y] = (prob_of_detection_nfv(prob, y_test), 
                        #false_alarm_rate_nfv(prob, y_test), 
                        #critical_success_index_nfv(prob, y_test))
        #result_df.rename({0: 'POD', 1: 'FAR', 2: 'CSI'}, inplace=True)
    return result_df

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, data, 3) #VFR

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, winter_data, 3) #VFR

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, data, 2) #MVFR

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, winter_data, 2) #MVFR

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, data, 1) #IFR

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, winter_data, 1) #IFR

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, data, 0) #LIFR

In [ ]:
%%time
test_accuracy_metrics(classifier_rf, winter_data, 0) #LIFR

In [ ]:
%%time
test_accuracy_metrics_nfv(classifier_rf, data) #Non-VFR

In [ ]:
%%time
test_accuracy_metrics_nfv(classifier_rf, winter_data) #Non-VFR

In [ ]:
%%time
(X, y_list) = data_split(data)
X_train, X_test, y_train, y_test = train_test_split(X, y_list['flight category 0'], train_size=0.7, random_state=42)
classifier_rf.fit(X_train, y_train)

imports = classifier_rf.feature_importances_

In [ ]:
importances = [(feature, sig) for feature, sig in zip(classifier_rf.feature_names_in_, imports)]
importances.sort(key = lambda x: x[1])
importances

In [ ]:
%%time
result = permutation_importance(
    classifier_rf, X_test, y_test, random_state=42, n_jobs=-1
)
#takes about 10 min

In [ ]:
importances = [(feature, sig) for feature, sig in zip(classifier_rf.feature_names_in_, result['importances_mean'])]
#importances.sort(key = lambda x: x[0].split('_')[1])
importances.sort(key = lambda x: -x[1])
importances = pd.DataFrame(importances)

In [ ]:
np.max(importances[1]), np.min(importances[1]), np.mean(importances[1]), np.std(importances[1])

In [ ]:
plt.plot(importances[1])

In [ ]:
with pd.option_context('display.max_rows', 999):
    print(importances)

In [ ]:
data

In [ ]:
condition_list = []
asos5 = read_metar('Data/BOS_5min.csv')
for _, metar_at_time in tqdm(asos5.iterrows()):
    vis = metar_at_time['vsby']
    cld_list = np.asarray(metar_at_time[['skyc1', 'skyc2', 'skyc3', 'skyc4']])
    hgt_list = np.asarray(metar_at_time[['skyl1', 'skyl2', 'skyl3', 'skyl4']])
    ovc_hgt = 100000
    bkn_hgt = 100000

    if 3 in list(cld_list):
        ovc_hgt = hgt_list[cld_list == 3]
        if len(ovc_hgt) > 1:
            ovc_hgt = np.min(ovc_hgt)
    if 2 in list(cld_list):
        bkn_hgt = hgt_list[cld_list == 2]
        if len(bkn_hgt) > 1:
            bkn_hgt = np.min(bkn_hgt)
    ceiling = np.min([ovc_hgt, bkn_hgt])

    if ceiling < 500 or vis < 1:
        conditions = 0
    elif ceiling < 1000 or vis < 3:
        conditions = 1
    elif ceiling < 3000 or vis < 5:
        conditions = 2
    else:
        conditions = 3
    condition_list.append(conditions)
asos5['conditions'] = condition_list

In [ ]:
asos5.to_csv('Data/BOS_5min.csv')